In [1]:
import os
from dotenv import load_dotenv
load_dotenv("main.env")  # Make sure you have GOOGLE_API_KEY in this file

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")



In [2]:
# loading docs


In [3]:
!pip show llama-index


Name: llama-index
Version: 0.12.44
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: 
Author-email: Jerry Liu <jerry@llamaindex.ai>
License-Expression: MIT
Location: /home/user/anaconda3/envs/llama_gemini/lib/python3.10/site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-question-gen-openai, llama-index-readers-file, llama-index-readers-llama-parse, nltk
Required-by: llama-hub


In [4]:
from pathlib import Path
from llama_index.readers.file import UnstructuredReader

reader = UnstructuredReader()
docs = reader.load_data(Path("docs/story.txt"))

In [5]:
# splitting into chuks


In [6]:
from llama_index.core.node_parser import SentenceSplitter


splitter = SentenceSplitter(
    chunk_size=384,         # balances granularity and context
    chunk_overlap=64,       # ensures smooth transitions
    paragraph_separator="\n\n"  # ideal for narrative/story formatting
)

nodes = splitter.get_nodes_from_documents(docs)

In [7]:
for i, node in enumerate(nodes[:3]):
    print(f"\n--- Chunk {i+1} ---")
    print(node.text[:500])  # preview up to 500 characters



--- Chunk 1 ---
Title: The Shadows of Arinvale

Prologue: Legends whispered of a city beneath the roots of the world — Arinvale. Few believed it. Fewer returned. But in the year 1421, strange lights began dancing across the Northern Forests, and the whispers turned into fearful prayers.

Chapter 1: The Letter Arin Windthorn, a scribe's apprentice in the coastal town of Evermist, receives a sealed letter with no name. Inside: a single sentence — "The truth sleeps beneath your name." Haunted by dreams of a sunken

--- Chunk 2 ---
Chapter 4: The Storm Ritual As storms begin to ravage the region, Nara performs the forgotten Rite of Echoes. In the heart of the tempest, Arin sees visions: his childhood rewritten, a burning village, and a sigil etched on his spine.

Chapter 5: The Betrayal Elgar returns — but not as an ally. Revealed as a Watcher in disguise, he tries to bind Arin using blood magic. Nara sacrifices her essence to sever the link, shattering the last storm crystal.

Chapter 6:

In [8]:
# generating embeeding with gemini

In [9]:
from llama_index.embeddings.gemini import GeminiEmbedding

embed_model = GeminiEmbedding(model="models/embedding-001")


/home/user/anaconda3/envs/llama_gemini/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_6481/2563350930.py:3: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  embed_model = GeminiEmbedding(model="models/embedding-001")


In [10]:
# pinecone 

In [11]:
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
from pinecone import Pinecone

# Define Gemini embedding model
embed_model = GeminiEmbedding(model="models/embedding-001")

# Connect to Pinecone
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pinecone_index = pc.Index("chatbot-index")
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

# Storage context
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# ✅ Build index (with embed_model explicitly passed)
index = VectorStoreIndex.from_documents(
    docs,
    storage_context=storage_context,
    embed_model=embed_model
)


/tmp/ipykernel_6481/2599069295.py:7: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  embed_model = GeminiEmbedding(model="models/embedding-001")
Upserted vectors: 100%|███████████████████████████| 1/1 [00:01<00:00,  1.65s/it]


In [12]:
from llama_index.llms.gemini import Gemini
from llama_index.core.query_engine import RetrieverQueryEngine  # ✅ from core



In [28]:
!pip install llama-index-llms-gemini


In [13]:
from llama_index.llms.gemini import Gemini

# Step 1: Create Gemini LLM
llm = Gemini(model="models/gemini-1.5-pro-latest")

# Step 2: Use the index's built-in query engine
query_engine = index.as_query_engine(llm=llm)

# Step 3: Ask a question
response = query_engine.query("What happened in Chapter 5?")
print(response)


/tmp/ipykernel_6481/741215637.py:4: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  llm = Gemini(model="models/gemini-1.5-pro-latest")


Elgar, the hunter Arin met earlier, reappears as a Watcher and attempts to use blood magic against Arin. Nara sacrifices herself to protect Arin, breaking the Watcher's hold and destroying the last storm crystal.



In [14]:
response = query_engine.query("Who is Nara?")
print(response)

Nara is the blind keeper of the Hollow Library, known for her riddles and for sacrificing herself to protect Arin.



In [15]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

query_engine_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="arinvale_story_query_engine",
        description="Useful for answering questions about characters, events, and themes in the Arinvale story.",
    ),
)


In [16]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools([query_engine_tool], llm=llm)

response = agent.query("What is the main theme of the story?")
print(response)


/home/user/anaconda3/envs/llama_gemini/lib/python3.10/site-packages/llama_index/core/agent/react/base.py:154: DeprecationWarning: Call to deprecated class ReActAgent. (ReActAgent has been rewritten and replaced by llama_index.core.agent.workflow.ReActAgent.

This implementation will be removed in a v0.13.0 and the new implementation will be promoted to the `from llama_index.core.agent import ReActAgent` path.

See the docs for more information: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return cls(
/home/user/anaconda3/envs/llama_gemini/lib/python3.10/site-packages/deprecated/classic.py:184: DeprecationWarning: Call to deprecated class AgentRunner. (AgentRunner has been deprecated and is not maintained.

This implementation will be removed in a v0.13.0.

See the docs for more information on updated agent usage: https://docs.llamaindex.ai/en/stable/understanding/agent/)
  return old_new1(cls, *args, **kwargs)


The main themes of the Arinvale story are identity and legacy, the power of memory, the cost of truth, and the interplay between magic and technology.


In [17]:
print("💬 Arinvale Chatbot is ready! (type 'exit' to quit)\n")

while True:
    query = input("You: ")
    if query.strip().lower() in {"exit", "quit"}:
        print("👋 Goodbye!")
        break

    try:
        response = query_engine.query(query)
        print(f"Bot: {response}\n")
    except Exception as e:
        print("⚠️ Error:", e)


💬 Arinvale Chatbot is ready! (type 'exit' to quit)



You:  who is nara?


Bot: Nara is the blind keeper of the Hollow Library, who speaks in riddles and ultimately sacrifices herself.




You:  what is supervised learning?


Bot: This query cannot be answered from the provided text, which describes the plot, characters, and setting of a fantasy novel.  There is no information about supervised learning.




You:  exit


👋 Goodbye!
